In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

  
    import numpy as np 
    import pandas as pd 
    import warnings
    warnings.filterwarnings("ignore")

In [1]:
def burau_and_bb(num_rows = None, nan_as_category = True):
       
    bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')
    bb = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv')
    
    bureau.drop(['CREDIT_CURRENCY'], axis = 1, inplace =True)
    
    bureau['CREDIT_TYPE'] = bureau['CREDIT_TYPE'].replace([
              'Microloan',
              'Loan for business development', 
              'Another type of loan',
              'Unknown type of loan', 
              'Loan for working capital replenishment',
              "Loan for purchase of shares (margin lending)",                                                
              'Cash loan (non-earmarked)', 
              'Real estate loan',
              "Loan for the purchase of equipment", 
              "Interbank credit", 
              "Mobile operator loan"], 'Rare')
    
    bureau.loc[bureau['DAYS_CREDIT_ENDDATE'] < -20000, 'DAYS_CREDIT_ENDDATE'] = np.nan
    bureau.loc[bureau['DAYS_ENDDATE_FACT'] < -20000, 'DAYS_ENDDATE_FACT'] = np.nan
    bureau.loc[bureau['DAYS_CREDIT_UPDATE'] < -20000, 'DAYS_CREDIT_UPDATE'] = np.nan
    
    bureau['NEW_DAYS_CREDIT_UPDATE_DIFF'] = bureau['DAYS_CREDIT'] - bureau['DAYS_CREDIT_UPDATE'] 
    bureau['NEW_EARLY_PAID'] = (bureau['DAYS_ENDDATE_FACT'] <= bureau['DAYS_CREDIT_ENDDATE']).astype('float') 
    bureau['NEW_OVERDUE_TO_CREDIT_SUM_RATIO'] = bureau['AMT_CREDIT_SUM_OVERDUE'] / bureau['AMT_CREDIT_SUM']
    bureau['NEW_OVERDUE_TO_DEBT_SUM_RATIO'] = bureau['AMT_CREDIT_SUM_OVERDUE'] / bureau['AMT_CREDIT_SUM_DEBT']
    bureau['NEW_DEBT_SUM_TO_CREDIT_SUM_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM']
    bureau['NEW_ANNUITY_TO_CREDIT_SUM_RATIO'] = bureau['AMT_ANNUITY'] / bureau['AMT_CREDIT_SUM']
    bureau['NEW_AMT_LIMIT_RATIO'] = bureau['AMT_CREDIT_SUM_LIMIT']/(1+bureau['AMT_CREDIT_SUM'])
    
    def one_hot_encoder(df, nan_as_category):
        original_columns = list(df.columns)
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
        df = pd.get_dummies(df, columns=categorical_columns,dummy_na=nan_as_category)
        new_columns = [c for c in df.columns if c not in original_columns]
        return df, new_columns
    
    bb, bb_cat = one_hot_encoder(bb, nan_as_category = True)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category = True)
    
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat: bb_aggregations[col] = ['mean']
        
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
    
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'DAYS_CREDIT_UPDATE': ['mean'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'AMT_ANNUITY': ['max', 'mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum'],
        'NEW_OVERDUE_TO_CREDIT_SUM_RATIO': ['min', 'max', 'mean'], 
        'NEW_OVERDUE_TO_DEBT_SUM_RATIO': ['min', 'max', 'mean'],
        'NEW_DEBT_SUM_TO_CREDIT_SUM_RATIO': ['min', 'max', 'mean'], 
        'NEW_ANNUITY_TO_CREDIT_SUM_RATIO': ['min', 'max', 'mean'],
        'NEW_DAYS_CREDIT_UPDATE_DIFF': ['min', 'max', 'mean'],
        'NEW_AMT_LIMIT_RATIO': ['min', 'max', 'mean'],
        'NEW_EARLY_PAID': ['mean']}

    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean', 'sum']
        
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BUREAU_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    
    del bureau, bb
    return bureau_agg
    

In [2]:
burau_and_bb()

,BUREAU_DAYS_CREDIT_MIN,BUREAU_DAYS_CREDIT_MAX,BUREAU_DAYS_CREDIT_MEAN,BUREAU_DAYS_CREDIT_ENDDATE_MIN,BUREAU_DAYS_CREDIT_ENDDATE_MAX,BUREAU_DAYS_CREDIT_ENDDATE_MEAN,BUREAU_DAYS_CREDIT_UPDATE_MEAN,BUREAU_CREDIT_DAY_OVERDUE_MAX,BUREAU_CREDIT_DAY_OVERDUE_MEAN,BUREAU_AMT_CREDIT_MAX_OVERDUE_MEAN,...,BUREAU_CREDIT_TYPE_Consumer credit_MEAN,BUREAU_CREDIT_TYPE_Consumer credit_SUM,BUREAU_CREDIT_TYPE_Credit card_MEAN,BUREAU_CREDIT_TYPE_Credit card_SUM,BUREAU_CREDIT_TYPE_Mortgage_MEAN,BUREAU_CREDIT_TYPE_Mortgage_SUM,BUREAU_CREDIT_TYPE_Rare_MEAN,BUREAU_CREDIT_TYPE_Rare_SUM,BUREAU_CREDIT_TYPE_nan_MEAN,BUREAU_CREDIT_TYPE_nan_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1572,-49,-735.000000,-1329.0,1778.0,82.428571,-93.142857,0,0.0,NaN,...,1.000000,7,0.000000,0,0.0,0,0.000000,0,0,0
100002,-1437,-103,-874.000000,-1072.0,780.0,-349.000000,-499.875000,0,0.0,1681.029,...,0.500000,4,0.500000,4,0.0,0,0.000000,0,0,0
100003,-2586,-606,-1400.750000,-2434.0,1216.0,-544.500000,-816.000000,0,0.0,0.000,...,0.500000,2,0.500000,2,0.0,0,0.000000,0,0,0
100004,-1326,-408,-867.000000,-595.0,-382.0,-488.500000,-532.000000,0,0.0,0.000,...,1.000000,2,0.000000,0,0.0,0,0.000000,0,0,0
100005,-373,-62,-190.666667,-128.0,1324.0,439.333333,-54.333333,0,0.0,0.000,...,0.666667,2,0.333333,1,0.0,0,0.000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,-2713,-483,-1667.076923,-2499.0,1363.0,-1232.333333,-1064.538462,0,0.0,6147.000,...,0.692308,9,0.230769,3,0.0,0,0.076923,1,0,0
456250,-1002,-760,-862.000000,-272.0,2340.0,1288.333333,-60.333333,0,0.0,0.000,...,0.666667,2,0.333333,1,0.0,0,0.000000,0,0,0
456253,-919,-713,-867.500000,-189.0,1113.0,280.500000,-253.250000,0,0.0,NaN,...,0.750000,3,0.250000,1,0.0,0,0.000000,0,0,0
